# Example 01

This example reproduces the results from Example 1 of from Quast & Wagner (2016)

The definition of the volume phase function and surface BRDF is as follows:

**Volume definition**

* optical depth ($\tau = 0.7$)
* single scattering albedo ($\omega = 0.3$)
* Volume phase function: Rayleigh phase function

**Surface definition**

* Cosine Lobe function with 10 coefficients

In [4]:
# imports
from rt1.rt1 import RT1 
from rt1.volume import Rayleigh   
from rt1.surface import CosineLobe 

In [5]:
# definition of volume and surface
V = Rayleigh(tau=0.7, omega=0.3)
SRF = CosineLobe(ncoefs=10, i=5)

**Imaging geometry (backscattering case)**

In [6]:
import numpy as np
# Specify imaging geometry
inc = np.arange(1.,89.,1.)  # specify incidence angle range [deg]
t_0 = np.deg2rad(inc)  # [rad]
# scattering angle; here the same as incidence angle, as backscatter
t_ex = t_0*1. 
# azimuth geometry angles
p_0 = np.ones_like(inc)*0. 
p_ex = np.ones_like(inc)*0. + np.pi  # 180 degree shift as backscatter

**Perform the simulations**

To perform the simulations, the RT model needs to estimate once coefficients. As these are the same for all imaging geometries, it makes sense to estimate these once and then transfer them to the subsequent calls.

In [8]:
# initial call to estimate coefficients (can be also omitted, but performance gets better like so)
I0=1.  # set incident intensity
R = RT1(I0, 0., 0., 0., 0., RV=V, SRF=SRF, geometry='mono')  
fn = R.fn  # store coefficients for faster iteration 


In [9]:
# do actual calculations with specifies geometries
R = RT1(I0, t_0, t_ex, p_0, p_ex, RV=V, SRF=SRF, fn=fn, geometry='mono')
Itot, Isurf, Ivol, Iint = R.calc()

In [ ]:
Plot these results

array([ 0.09511944,  0.09431831,  0.09299915,  0.0911856 ,  0.08890987,
        0.0862119 ,  0.08313833,  0.07974129,  0.0760771 ,  0.07220482,
        0.06818489,  0.06407761,  0.05994185,  0.05583367,  0.05180523,
        0.04790374,  0.04417066,  0.04064107,  0.03734327,  0.03429857,
        0.03152133,  0.02901914,  0.02679329,  0.02483922,  0.02314732,
        0.02170362,  0.02049071,  0.01948856,  0.01867545,  0.01802879,
        0.01752588,  0.01714467,  0.01686429,  0.01666555,  0.01653132,
        0.01644668,  0.01639908,  0.01637832,  0.01637639,  0.0163873 ,
        0.01640684,  0.01643222,  0.01646177,  0.01649461,  0.01653035,
        0.01656883,  0.01660989,  0.01665339,  0.01669915,  0.016747  ,
        0.01679674,  0.01684817,  0.01690106,  0.0169552 ,  0.01701033,
        0.0170662 ,  0.01712256,  0.01717914,  0.01723564,  0.01729179,
        0.01734727,  0.01740178,  0.01745499,  0.01750659,  0.01755624,
        0.0176036 ,  0.01764836,  0.01769018,  0.01772876,  0.01